In [25]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, make_scorer, mean_squared_error
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn import preprocessing

import warnings
warnings.filterwarnings("ignore")

In [26]:
# Cargar los datos
path = "/code/Python/Quiz_1/"
dataset = pd.read_csv(path + "iris_dataset.csv", header=0, sep=";", decimal=",")
prueba = pd.read_csv(path + "iris_prueba.csv", header=0, sep=";", decimal=",")

# Definir covariables y objetivo
covariables = [x for x in dataset.columns if x != "y"]
X = dataset[covariables]
y = dataset["y"].values.ravel()

X_nuevo = prueba[covariables]
y_nuevo = prueba["y"].values.ravel()

In [27]:
# División del conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=123)

# Escalado de los datos
scaler = preprocessing.StandardScaler()
X_train_Z = scaler.fit_transform(X_train)
X_test_Z = scaler.transform(X_test)

In [28]:
# Modelo y GridSearchCV para los datos de entrenamiento
modelo1 = LogisticRegression(random_state=123)
parametros = {'C': np.arange(0.1, 5.1, 0.1)}
grilla = GridSearchCV(estimator=modelo1, param_grid=parametros, cv=5, scoring=make_scorer(accuracy_score), n_jobs=-1)
grilla.fit(X_train_Z, y_train)

# Obtener mejores parámetros
best_params1 = grilla.best_params_


In [29]:
# Modelo y GridSearchCV para los datos de prueba (aunque no es común hacer esto)
modelo2 = LogisticRegression(random_state=123)
grilla2 = GridSearchCV(estimator=modelo2, param_grid=parametros, cv=5, scoring=make_scorer(accuracy_score), n_jobs=-1)
grilla2.fit(X_test_Z, y_test)

# Obtener mejores parámetros
best_params2 = grilla2.best_params_

In [30]:
# Predicciones y cálculo de la precisión
y_hat_test = grilla.predict(X_test_Z)
y_hat_train = grilla2.predict(X_train_Z)

u1 = accuracy_score(y_test, y_hat_test)
u2 = accuracy_score(y_train, y_hat_train)

In [31]:
if np.abs(u1 - u2) < 0.1:
    # Entrenar un modelo completo con todo el conjunto de datos
    modelo_completo = LogisticRegression(random_state=123)
    grilla_completa = GridSearchCV(estimator=modelo_completo, param_grid=parametros, cv=5, scoring=make_scorer(accuracy_score), n_jobs=-1)

    # Escalar todos los datos
    X_Z = scaler.fit_transform(X)

    # Entrenar el modelo con todos los datos
    grilla_completa.fit(X_Z, y)
else:
    print("No cumple la condición np.abs(u1 - u2) < 0.1, continúa sin GridSearchCV.")

    modelo_completo = LogisticRegression(random_state=123)

    # Escalar todos los datos
    X_Z = scaler.fit_transform(X)

    # Entrenar el modelo con todos los datos
    modelo_completo.fit(X_Z, y)
    grilla_completa = modelo_completo  # Usar el modelo sin GridSearchCV

In [32]:
# Acurracy 0.33333 si
# X_nuevo_Z = X_nuevo
# y_hat_nuevo = grilla_completa.predict(X_nuevo_Z)

# Acurracy 1 si
X_nuevo_Z = scaler.transform(X_nuevo)
y_hat_nuevo = grilla_completa.predict(X_nuevo_Z)

accuracy = accuracy_score(y_nuevo, y_hat_nuevo)

# Mostrar la precisión en el conjunto de prueba nuevo
print(f"Precisión en el nuevo conjunto de datos: {accuracy}")


Precisión en el nuevo conjunto de datos: 1.0
